# Proyecto NLP Quijote

**Objetivo**
Creación de un modelo de Deep Learning entrenado con los primeros **50 capitulos del Quijote**, para crear una contextualización artificial del contenido del libro y de esta manera poder predecir un nuevo texto en función a unas palabras dadas.

**Requisitos**
* Python 3.8
* Tensorflow 2.x

**Pasos de creación**
* Adquirir el libro del Quijote en formato digital. https://www.gutenberg.org/ebooks/search/?query=quijote&submit_search=Go%21
* Tratamiento de los datos
* Creación del modelo usando redes LSTM (Large Short Term Memory)
* Entrenamiento
* Resultados y validación
* Prediccón
* Salvar modelo

In [ ]:
#Cargamos las librerias correspondientes
import tensorflow as tf

from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dropout
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint
import numpy as np
import re

from codecarbon import EmissionsTracker

### Tokenizer
Como la palabra sugiere tokenizar significa dividir la oración en una serie de tokens o en palabras simples, podemos decir que cada vez que hay un espacio en una oración agregamos una coma entre ellos para que nuestra oración se divida en tokens y cada palabra tenga un valor único de un número entero.

In [ ]:
tokenizer = Tokenizer()

data = open('Dataset/quijote_Lite.txt', encoding="utf8").read()
#data = open('/content/drive/My Drive/Colab Notebooks/Datasets/Vuelta_al_mundo.txt').read()

#Limpiar de simbolos 
data = re.sub('[^a-zA-Z0-9á-ú\¿\?\n\.]', ' ', data)
corpus = data.lower().split("\n")

#Mostramos el cuerpo del texto ya limpio
#print(corpus)
corpus = sorted(list(set(corpus)))

tokenizer.fit_on_texts(corpus)
total_words = len(tokenizer.word_index) + 1



### Resultado despues de realizar los **tokens**

In [ ]:
#print(tokenizer.word_index)
#print(total_words)
#print(corpus)
#print(len(corpus))

#### Preparación de los datos
Para mejorar el entrenamiento del modelo y poder tener más datos a partir de los obtenidos, se realiza una tecnica llamda **secuencia**. Que consiste en dividir cada oración en una más pequeña en forma de escalera, de esta forma se hará una predicción de entrenamiento por cada subdivisión de esa oración.


Posteriormente se realiza un **Padding**. Es un método para convertir la matriz de enteros de longitud variable en una longitud fija, ya sea truncando (si la longitud es mayor que la longitud_máxima definida que trunca la matriz) o rellenando (si la longitud es más corta que la longitud_máxima, rellene la matriz con 0).

<img src="images/Padding.png">

In [ ]:
input_sequences = []
#Marcamos los tokens por cada frase
for line in corpus:
	token_list = tokenizer.texts_to_sequences([line])[0]
	#Creamos frases más pequeñas en función a al original
	#for i in range(1, len(token_list)):
	for i in range(1, len(token_list)): #usamos cada frase para aumentar el train en modo de escalera
		n_gram_sequence = token_list[:i+1]
		input_sequences.append(n_gram_sequence)

# pad sequences 
#Buscamos la frase más larga y las igualamos todas con ceros en la parte de adelante
max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

# create predictors and label
#Cogemos el ultimo valor como etiqueta (y) y el resto como (x)
xs, labels = input_sequences[:,:-1],input_sequences[:,-1]

ys = tf.keras.utils.to_categorical(labels, num_classes=total_words)

**Mostramos algunos ejemplos del proceso**

In [ ]:
print(xs[5])
print(ys[5])
print(ys.shape)
print(xs.shape)

### Creación del modelo

* La primera capa incluye el número de palabras a entrenar y la salida de predicción que queremos mostrar, en este caso se hará una predicción de 50 palabras en función a la dada.

* Usamos como optimizador Adam, aunque tambien se optienen buenos resultados con RMSprop

In [ ]:
model = Sequential()
model.add(Embedding(total_words, 50, input_length=max_sequence_len-1))
#model.add(Bidirectional(LSTM(128)))
#model.add(Dropout(0.2))
model.add(LSTM(100, return_sequences=True))
model.add(LSTM(100))
model.add(Dense(100,activation='relu'))
model.add(Dense(total_words, activation='softmax'))
adam = Adam(lr=0.001)
rms=RMSprop(learning_rate=0.01)

model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
#earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=5, verbose=0, mode='auto')
#filepath="/content/drive/My Drive/Colab Notebooks/Projects/QuijoteNLP/model/weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"
filepath="model/weights-QuijoteNLP.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='accuracy', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

model.summary()



In [ ]:
# Cargar pesos del modelo (evitar entrenamiento)
model.load_weights("model/weights-QuijoteNLP.hdf5")

### Analizar el consumo energético e impacto ambiental del enetrenamiento del modelo mediante **CodeCarbon**
CodeCarbon es un proyecto de código abierto diseñado para medir la huella de carbono de los algoritmos de inteligencia artificial.

#### ¿Cómo funciona?

* Monitoreo Transparente: CodeCarbon funciona en segundo plano mientras entrenas tus modelos de IA. Recopila datos sobre el consumo de energía y utiliza información sobre la ubicación del servidor y los tipos de fuentes de energía para calcular la huella de carbono.

* Facilidad de Uso: Integrar CodeCarbon en proyectos de IA es sencillo. Esto lo hace accesible para investigadores, desarrolladores y educadores que deseen medir el impacto ambiental de su trabajo.

* Resultados Accionables: La herramienta no solo mide la huella de carbono, sino que también proporciona insights y recomendaciones para reducir el impacto ambiental, fomentando prácticas de IA más sostenibles.

In [ ]:
with EmissionsTracker() as tracker:
    history = model.fit(xs, ys, epochs=70, batch_size=128, verbose=1,callbacks=callbacks_list)

### Visualizar datos de consumo energético del modelo

In [ ]:
!carbonboard --filepath="emissions.csv" --port=3333

Dash is running on http://127.0.0.1:3333/

 * Serving Flask app 'codecarbon.viz.carbonboard'
 * Debug mode: off
 * Running on http://127.0.0.1:3333
Press CTRL+C to quit
127.0.0.1 - - [16/Jan/2024 12:34:49] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [16/Jan/2024 12:34:49] "GET /_dash-component-suites/dash/deps/polyfill@7.v2_12_1m1692783890.12.1.min.js HTTP/1.1" 200 -
127.0.0.1 - - [16/Jan/2024 12:34:49] "GET /_dash-component-suites/dash/deps/react@16.v2_12_1m1692783890.14.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [16/Jan/2024 12:34:49] "GET /_dash-component-suites/dash_bootstrap_components/_components/dash_bootstrap_components.v1_5_0m1705348573.min.js HTTP/1.1" 200 -
127.0.0.1 - - [16/Jan/2024 12:34:49] "GET /_dash-component-suites/dash/deps/react-dom@16.v2_12_1m1692783890.14.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [16/Jan/2024 12:34:49] "GET /_dash-component-suites/dash/dash-renderer/build/dash_renderer.v2_12_1m1692783890.min.js HTTP/1.1" 200 -
127.0.0.1 - - [16/Jan/2024 12:34:49] "GET /_dash-co

### Análisis de eficacia del modelo

In [ ]:
import matplotlib.pyplot as plt


def plot_graphs(history, string):
  plt.plot(history.history[string])
  plt.xlabel("Epochs")
  plt.ylabel(string)
  plt.show()

In [ ]:
plot_graphs(history, 'accuracy')


## Predicción de un texto nuevo
* Le damos unas palabras para que pueda crear unas lineas contextualizadas en el Quijote.

In [ ]:
seed_text = "leer libros de caballerías"
next_words = 50
  
for _ in range(next_words):
	token_list = tokenizer.texts_to_sequences([seed_text])[0]
	token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
	predicted = model.predict_classes(token_list, verbose=0)
	output_word = ""
	for word, index in tokenizer.word_index.items():
		if index == predicted:
			output_word = word
			break
	seed_text += " " + output_word
print(seed_text)

#### Se guarda la predicción en un archivo de texto

In [ ]:
file = open("prediction.txt", "w") 
file.write(seed_text) 
file.close() 

In [ ]:
e = model.layers[0]
weights = e.get_weights()[0]
print(weights.shape) # shape: (vocab_size, embedding_dim)

In [ ]:
#cantidad de palabras unicas
len(tokenizer.word_index.keys())

In [ ]:
#Convertimos el diccionario en una lista, obteniendo solo las palabras
words_list = [(k) for k in tokenizer.word_index.keys()]

In [ ]:
import io

out_v = io.open('vecs.tsv', 'w', encoding='utf-8')
out_m = io.open('meta.tsv', 'w', encoding='utf-8')

for num, word in enumerate(words_list):
  vec = weights[num+1] # skip 0, it's padding.
  out_m.write(word + "\n")
  out_v.write('\t'.join([str(x) for x in vec]) + "\n")
out_v.close()
out_m.close()

try:
  from google.colab import files
except ImportError:
   pass
else:
  files.download('vecs.tsv')
  files.download('meta.tsv')